# 📰 與文件問答

<img src="https://www.notion.so/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F82622d80-8c8a-4a92-a5a7-7751ab86c495%2FUntitled.png?table=block&id=2c5011af-cbed-49fc-b986-98883b204326&spaceId=3d67fbfd-2c46-4608-851e-549759470329&width=2000&userId=42adc2a8-d4a8-44c6-a20d-b52ec23ba594&cache=v2" width=640 height=400>

限制:
* 不能讀取過長的文本(chain_type=stuff)
* 有時候會回復非 prompt template 之外的內容
* 文本僅含有一種產品規格


prompt template:
```
Use the following pieces of context to answer the question, 
if you don't know the answer, leave it blank don't try to make up an answer.
        {context}
Question: {question}
Answer in JSON representations
```

query content
```
What is the display specifications?
include: company_name, 
        product_name, 
        size_inch,
        resolution,contrast,
        operation_temperature,
        sunlight_readable,
        antiglare,
```

In [7]:
import docsGPT
import json

In [8]:
example_data1 = "example_data/rxi-web-panel-emerson.pdf"
example_data2 = "example_data/HEM-6232T Manual - Omron Healthcare.pdf"
example_data3 = "example_data/10-04-580-SPC - Wall-Smart.pdf"


docs_result = docsGPT.chat_document(example_data2)

In [9]:
json.loads(docs_result)

{'company_name': 'OMRON',
 'product_name': 'Wrist Blood Pressure Monitor',
 'size_inch': 'N/A',
 'resolution': 'N/A',
 'contrast': 'N/A',
 'operation_temperature': 'N/A',
 'sunlight_readable': 'N/A',
 'antiglare': 'N/A'}

In [10]:
docs_result = docsGPT.chat_document(example_data3)
json.loads(docs_result)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'company_name': 'Control4',
 'product_name': 'T3 Series 7" Tabletop Touch Screen',
 'size_inch': 7,
 'resolution': '1280 × 800',
 'contrast': 'N/A',
 'operation_temperature': '32 ~ 104˚F (0˚ ~ 40˚C)',
 'sunlight_readable': 'No',
 'antiglare': 'No'}

# 🕵️ 整合搜尋引擎以及LLM

使LLM具備推理能力，能夠以一連串的行為，推理出最後答案

In [4]:
import searchGPT

example_text = """
            Can you provide the company name manufacturer of portable detector wihch have monitor on it?
            answering me at least 3 companies names and return the company names with following format
            company_list= [company_name1, company_name2, company_name3].
            """
searchGPT.search_gpt(example_text)


WARNING! api_key is not default parameter.
                    api_key was transferred to model_kwargs.
                    Please confirm that api_key is what you intended.




> Entering new AgentExecutor chain...
 I should search for companies that make portable detectors with monitors.
Action: Search
Action Input: "portable detector with monitor"
Observation: Mini Portable CO2 Detector, Aiment 3-in-1 Lightweight Carbon Dioxide Detector Air Quality Monitor Temperature Humidity Air Analyzer Digital CO2 Meter for ...
Thought: I should look for more companies that make portable detectors with monitors.
Action: Search
Action Input: "portable detector with monitor companies"
Observation: At PK Safety, we sell dozens of high-quality portable multi gas monitors and single gas monitors, so you can protect yourself from a range of ...
Thought: I should look for more companies that make portable detectors with monitors.
Action: Search
Action Input: "portable detector with monitor manufacturers"
Observation: ChemLogic CLPx Portable Gas Detector. $7,671.00 ; XPS-7II Portable Electrochemical Gas Detector. $2,430.00 ; IsoSense Personal MDI Detection Kit.
Thought: I now

# 💁 [QA Over Documents](https://python.langchain.com/docs/use_cases/question_answering/)

<img src="https://python.langchain.com/assets/images/qa_flow-9fbd91de9282eb806bda1c6db501ecec.jpeg" height=360>

## Quick Start
To give you a sneak preview, the above pipeline can be all be wrapped in a single object: `VectorstoreIndexCreator`. 
<br>
Suppose we want a QA app over this blog post. We can create this in a few lines of code:


In [1]:
from langchain.document_loaders import WebBaseLoader
from langchain.indexes import VectorstoreIndexCreator

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
index = VectorstoreIndexCreator().from_loaders([loader])

In [2]:
index.query("What is Task Decomposition?")


' Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. It can be done using LLM with simple prompting, task-specific instructions, or human inputs. Tree of Thoughts (Yao et al. 2023) is an example of a task decomposition technique that explores multiple reasoning possibilities at each step and generates a tree structure.'

## 1. Load Data


In [3]:
from langchain.document_loaders import WebBaseLoader


# Load Data
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

## 2. Split

In [4]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

## 3. Vector Store

In [6]:
# Store
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

## 4. Retrieve

In [7]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [32]:
from langchain.retrievers import SVMRetriever

svm_retriever = SVMRetriever.from_documents(all_splits,OpenAIEmbeddings())
docs_svm=svm_retriever.get_relevant_documents(question)
len(docs_svm)

AttributeError: type object 'SVMRetriever' has no attribute 'from_documents'

## 5. Generate

In [10]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever())
qa_chain({"query": question})

{'query': 'What are the approaches to Task Decomposition?',
 'result': 'The approaches to task decomposition include:\n\n1. Simple prompting: This approach involves using simple prompts or questions to guide the agent in breaking down a task into smaller subgoals. For example, the agent can be prompted with "Steps for XYZ. 1." or "What are the subgoals for achieving XYZ?"\n\n2. Task-specific instructions: In this approach, task-specific instructions are provided to the agent to guide the decomposition process. For example, if the task is to write a novel, the agent can be instructed to "Write a story outline." This helps the agent in understanding the specific requirements of the task and breaking it down accordingly.\n\n3. Human inputs: Another approach to task decomposition is to involve human inputs. This can be done by seeking guidance or feedback from humans who have expertise in the task domain. Human inputs can provide valuable insights and help in breaking down complex tasks ef